# 2a - Sequence to Sequence Learning with Neural Networks

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](assets/seq2seq1.png)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](assets/seq2seq4.png)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Import Libraries



In [ ]:
#!pip install -U spacy

In [ ]:
#!pip install -U swifter

In [2]:
# code here

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#data_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/Data/NLP')
project_folder = Path('/content/drive/MyDrive/Research/NLP/pytorch-seq2seq')

In [5]:
torchtext.__version__, torch.__version__, torch.cuda.is_available(), spacy.__version__

('0.11.0', '1.10.0+cu111', True, '2.2.4')

# Set Seeds

In [6]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load Tokenized Data

In [20]:
df_train = pd.read_pickle(project_folder/'df_train_en_de.pickel')
df_test =  pd.read_pickle(project_folder/'df_test_en_de.pickel')
df_valid = pd.read_pickle(project_folder/'df_valid_en_de.pickel')


In [21]:
df_train

,source_tokens,target_tokens,source_tokens_reverse
0,"[zwei, junge, weiße, männer, sind, i, m, freie...","[two, young, ,, white, males, are, outside, ne...","[., büsche, vieler, nähe, der, in, freien, m, ..."
1,"[mehrere, männer, mit, schutzhelmen, bedienen,...","[several, men, in, hard, hats, are, operating,...","[., antriebsradsystem, ein, bedienen, schutzhe..."
2,"[ein, kleines, mädchen, klettert, in, ein, spi...","[a, little, girl, climbing, into, a, wooden, p...","[., holz, aus, spielhaus, ein, in, klettert, m..."
3,"[ein, mann, in, einem, blauen, hemd, steht, au...","[a, man, in, a, blue, shirt, is, standing, on,...","[., fenster, ein, putzt, und, leiter, einer, a..."
4,"[zwei, männer, stehen, am, herd, und, bereiten...","[two, men, are, at, the, stove, preparing, foo...","[., zu, essen, bereiten, und, herd, am, stehen..."
...,...,...,...
28995,"[., wand, verschnörkelten, einer, hinter, schr...","[a, woman, behind, a, scrolled, wall, is, writ...","[eine, frau, schreibt, hinter, einer, verschnö..."
28996,"[., kletterwand, einer, an, übt, bergsteiger, ...","[a, rock, climber, practices, on, a, rock, cli...","[ein, bergsteiger, übt, an, einer, kletterwand..."
28997,"[., hauses, einem, vor, straße, einer, auf, ar...","[two, male, construction, workers, are, workin...","[zwei, bauarbeiter, arbeiten, auf, einer, stra..."
28998,"[., fassade, einer, vor, wagen, einem, mit, ju...","[an, elderly, man, sits, outside, a, storefron...","[ein, älterer, mann, sitzt, mit, einem, jungen..."


In [22]:
df_valid

,source_tokens,target_tokens,source_tokens_reverse
0,"[eine, gruppe, von, männern, lädt, baumwolle, ...","[a, group, of, men, are, loading, cotton, onto...","[lastwagen, einen, auf, baumwolle, lädt, männe..."
1,"[ein, mann, schläft, in, einem, grünen, raum, ...","[a, man, sleeping, in, a, green, room, on, a, ...","[., sofa, einem, auf, raum, grünen, einem, in,..."
2,"[ein, junge, mit, kopfhörern, sitzt, auf, den,...","[a, boy, wearing, headphones, sits, on, a, wom...","[., frau, einer, schultern, den, auf, sitzt, k..."
3,"[zwei, männer, bauen, eine, blaue, eisfischerh...","[two, men, setting, up, a, blue, ice, fishing,...","[auf, see, zugefrorenen, einem, auf, eisfische..."
4,"[ein, mann, mit, beginnender, glatze, ,, der, ...","[a, balding, man, wearing, a, red, life, jacke...","[., boot, kleinen, einem, in, sitzt, ,, trägt,..."
...,...,...,...
1009,"[., her, zuckerwatte, mann, ein, stellt, jahrm...","[at, some, sort, of, carnival, ,, a, man, is, ...","[bei, einer, art, jahrmarkt, stellt, ein, mann..."
1010,"[., bus, einem, vor, steht, polizisten, von, g...","[a, bunch, of, police, officers, are, standing...","[eine, gruppe, von, polizisten, steht, vor, ei..."
1011,"[., hindurch, brillengläser, ihre, durch, blic...","[a, elderly, white, -, haired, woman, is, look...","[eine, ältere, weißhaarige, frau, sieht, in, i..."
1012,"[., freien, m, i, telefonzellen, an, stehen, m...","[two, men, are, standing, at, telephone, booth...","[zwei, männer, stehen, an, telefonzellen, i, m..."


In [23]:
df_test

,source_tokens,target_tokens,source_tokens_reverse
0,"[ein, mann, mit, einem, orangefarbenen, hut, ,...","[a, man, in, an, orange, hat, starring, at, so...","[., anstarrt, etwas, der, ,, hut, orangefarben..."
1,"[ein, boston, terrier, läuft, über, saftig, -,...","[a, boston, terrier, is, running, on, lush, gr...","[., zaun, weißen, einem, vor, gras, grünes, -,..."
2,"[ein, mädchen, in, einem, karateanzug, bricht,...","[a, girl, in, karate, uniform, breaking, a, st...","[., tritt, einem, mit, stock, einen, bricht, k..."
3,"[fünf, leute, in, winterjacken, und, mit, helm...","[five, people, wearing, winter, jackets, and, ...","[., hintergrund, m, i, schneemobilen, mit, sch..."
4,"[leute, reparieren, das, dach, eines, hauses, .]","[people, are, fixing, the, roof, of, a, house, .]","[., hauses, eines, dach, das, reparieren, leute]"
...,...,...,...
995,"[., stehen, herum, sie, um, personen, andere, ...","[marathon, runners, are, racing, on, a, city, ...","[marathonläufer, laufen, auf, einer, städtisch..."
996,"[., fahrradfahren, beim, sonnenhut, einen, trä...","[asian, woman, wearing, a, sunhat, while, ridi...","[asiatische, frau, trägt, einen, sonnenhut, be..."
997,"[., bäumen, zwei, bei, boden, dem, auf, spiele...","[some, children, are, outside, playing, in, th...","[ein, paar, kinder, sind, i, m, freien, und, s..."
998,"[., videospiel, ein, spielt, mann, älterer, ein]","[an, older, man, is, playing, a, video, arcade...","[ein, älterer, mann, spielt, ein, videospiel, .]"


## Build Vocab

In [24]:
def create_vocab(text, min_freq, specials):
    #code here


In [25]:
source_vocab = # code here

In [26]:
len(source_vocab)

7874

In [27]:
pd.DataFrame(source_vocab.get_stoi().items(), columns=['tokens', 'index']).sort_values(by = ['index'])

,tokens,index
3373,<unk>,0
936,<BOS>,1
2372,<EOS>,2
3624,<PAD>,3
2772,zwei,4
...,...,...
4,geländefahrzeugs,7869
3,sonnentag,7870
2,lederoberteil,7871
1,reisfeld,7872


In [28]:
# check index of unknown word - it should be zero
source_vocab['abracdabra']

0

In [29]:
target_vocab = create_vocab(df_train['target_tokens'], 2, ['<unk>', '<BOS>', '<EOS>', '<PAD>'])

In [30]:
len(target_vocab)

5893

# Create Dataset and Dataloader

In [31]:
class EngGerman(Dataset):
    # code here


In [45]:
trainset =  # code here
testset =  # code here
validset =  # code here

In [46]:
trainset.__getitem__(0)

(['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'm',
  'i',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'])

In [47]:
len(trainset), len(testset), len(validset)

(29000, 1000, 1014)

In [48]:
# return a list for index of tokens in following order [<BOS> ,text tokens ,<EOS>]
def text_transform (my_vocab, text):
    # code here


In [49]:
text = trainset.__getitem__(12)[1]
print(text)
text_transform(target_vocab, text)

['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']


tensor([ 1, 21, 90, 91, 92, 21, 93, 91,  9, 94,  2])

In [50]:
def collate_batch(batch):
    # code here


In [51]:
batch_size = 2

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                              collate_fn=collate_batch)

In [52]:
for source, target in train_loader:
  print(source)
  break

tensor([[  1,   1],
        [ 22,   4],
        [ 29,  74],
        [ 12,  40],
        [ 30,  12],
        [363,  13],
        [173, 899],
        [101,  17],
        [386,   2],
        [ 55,   3],
        [ 17,   3],
        [  2,   3]])


In [53]:
batch_size = 128

train_loader = # code here
valid_loader = # code here
test_loader =  # code here

## Building the Seq2Seq Model

## Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](assets/seq2seq5.png)

In [54]:
class Encoder(nn.Module):


## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [55]:
class Decoder(nn.Module):


## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [56]:
class Seq2Seq(nn.Module):

        
 

# Training the Seq2Seq Model

The rest of this tutorial is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [57]:
INPUT_DIM = len(source_vocab)
OUTPUT_DIM = len(target_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [58]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, 0, 0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7874, 256, padding_idx=3)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256, padding_idx=3)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We also define a function that will calculate the number of trainable parameters in the model.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [59]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,225,157 trainable parameters


We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.

In [60]:
optimizer = optim.Adam(model.parameters())

Next, we define our loss function. The `CrossEntropyLoss` function calculates both the log softmax as well as the negative log-likelihood of our predictions. 

Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [61]:
TRG_PAD_IDX = target_vocab['<PAD>']
TRG_PAD_IDX

3

In [62]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Next, we'll define our training loop. 

First, we'll set the model into "training mode" with `model.train()`. This will turn on dropout (and batch normalization, which we aren't using) and then iterate through our data iterator.

As stated before, our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Here, when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we slice off the first column of the output and target tensors as mentioned above
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [63]:
def train(model, iterator, optimizer, criterion, clip):
    


Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used).

We use the `with torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up. 

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [64]:
def evaluate(model, iterator, criterion):
    
    


Next, we'll create a function that we'll use to tell us how long an epoch takes.

In [65]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We can finally start training our model!

At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called `state_dict` in PyTorch). Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss. 

We'll be printing out both the loss and the perplexity at each epoch. It is easier to see a change in perplexity than a change in loss as the numbers are much bigger.

In [66]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model_1a.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 29s
	Train Loss: 5.056 | Train PPL: 156.950
	 Val. Loss: 5.016 |  Val. PPL: 150.750
Epoch: 02 | Time: 0m 28s
	Train Loss: 4.382 | Train PPL:  80.004
	 Val. Loss: 4.769 |  Val. PPL: 117.819
Epoch: 03 | Time: 0m 29s
	Train Loss: 4.079 | Train PPL:  59.100
	 Val. Loss: 4.570 |  Val. PPL:  96.531
Epoch: 04 | Time: 0m 29s
	Train Loss: 3.777 | Train PPL:  43.678
	 Val. Loss: 4.303 |  Val. PPL:  73.928
Epoch: 05 | Time: 0m 29s
	Train Loss: 3.479 | Train PPL:  32.423
	 Val. Loss: 4.100 |  Val. PPL:  60.358
Epoch: 06 | Time: 0m 28s
	Train Loss: 3.202 | Train PPL:  24.576
	 Val. Loss: 3.961 |  Val. PPL:  52.490
Epoch: 07 | Time: 0m 29s
	Train Loss: 2.950 | Train PPL:  19.114
	 Val. Loss: 3.904 |  Val. PPL:  49.606
Epoch: 08 | Time: 0m 29s
	Train Loss: 2.724 | Train PPL:  15.236
	 Val. Loss: 3.841 |  Val. PPL:  46.594
Epoch: 09 | Time: 0m 29s
	Train Loss: 2.522 | Train PPL:  12.457
	 Val. Loss: 3.772 |  Val. PPL:  43.475
Epoch: 10 | Time: 0m 29s
	Train Loss: 2.329 | Train PPL

We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [67]:
model.load_state_dict(torch.load('tut1-model_1a.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.677 | Test PPL:  39.534 |


In the following notebook we'll implement a model that achieves improved test perplexity, but only uses a single layer in the encoder and the decoder.